# Tanzania Water Pump Classifier

The Tanzanian government has worked in conjunction with for-profit and non-profit organizations to build water pumps across Tanzania to provide its denizens with potable water. These pumps need maintenance to continue to operate. It is expensive, time-consuming, and inefficient to send repair teams, and tools and parts, only when the government receives reports of a faulty pump. The purpose of this project is to create a machine learning algorithm that can predict which pumps need repair or replacement to save the time and money of the Tanzanian government and its cooperative organizations.

This data was from a private Kaggle competition held by BloomTech for its DS36 Data Science cohort; its data mirrors that of the community Kaggle competition.

This documentation presents the project narrative in a CRISP-DM process style.

## Import Libraries

All libraries used throughout the notebook will be initialized here.

In [1]:
import numpy as np
import pandas as pd

## Business Understanding

We want to create a machine learning algorithm that can predict which pumps are non-functional. Let's expand on this goal by further developing our understanding of the stakeholder's needs.

### Goal

The Tanzanian government wants to keep the pumps in working order to provide a reliable source of water for its population. They have "hired" us to build a model that can turn accessible water pump data into three classes of predictions:

1. Functional (the pump is working)
2. Functional needs repair (the pump needs repair)
3. Non functional (the pump needs replacement)

With the predictions, the Tanzanian government will send the necessary tools and teams to the pumps needing repair, and replacement teams to the broken pumps.

### Additional Factors and Considerations

We should be prepared for the inevitability that our model is unlikely predict with perfect accuracy. An imperfect classification model is subject to Type I and Type II errors. In the case of water pump classification:

- Type I, False Positive: The model predicts that the pump needs repair or replacement when it is in working condition
- Type II, False Negative: The model predicts that the pump is in working condition when it needs repair or replacement

Both types of errors should be avoided, but a Type II error is much more pressing. The worst case scenario in a Type II error would be when the model predicts a pump to be in working condition when it is broken and needs replacement. We're dealing with water, which is necessary to live. A Type I error would result in a waste of resources (sending repair teams and tools to a working pump), but a Type II error may result in a loss of lives!

We could tune our model to eliminate Type II errors, but this would result in more Type I errors; more pumps that are functional would be marked as needing repair or replacement, which would be an unnecessary drain on repair resources.

If we were able to meet with the stakeholder (the Tanzanian government) we would want to clarify how we should prioritize the reduction of these errors.

We will assume that the stakeholder prefers to begin with a model that is as accurate as possible, and make adjustments to account for these errors when we have an MVP (minimum viable product).

### Accuracy Metric

For the MVP, we will focus on building a model that is as accurate as possible. There are several metrics we can use, which have their own benefits and drawbacks:

1. Basic accuracy metric
    - Pros: simple
    - Cons: does not account for precision and recall (measures impact of Type I and Type II errors)
2. Weighted F1 Score
    - Pros: accounts for precision and recall
    - Cons: does not directly account for poor scores in one classification (averages all classification scores)
3. Multiclass ROC AUC
    - Pros: accounts for precision and recall; accounts for poor scores in one classification; easiest to visualize; can average scores for one final metric
    - Cons: introduces subjective assessment of success (multiple measures are weighed, assessor must rank measure importance)

In accordance with the idea that the stakeholder may want to make adjustments to the model to account for errors, multiclass ROC AUC is the best choice. We will implement a 'One vs Rest' style of multiclass ROC AUC, which will give us a total of three graphs, with each graph comparing one feature to the other two. Changes to precision and recall for each class will be most visible while using this metric. In addition, if we wanted a single measure of accuracy, we retain the option to consolidate the scores for each graph into a simple or weighted average.

### Project Plan

Let's outline a plan for how we will approach this project.

1. Data Understanding
    - *Identify data surface qualities*: What is our data format? How many records do we have to train our model?
    - *Verify data quality*: How dirty is the data? Are there any non-existent values? Incorrect values or types?
    - *Understand features*: What kinds of data are available?
    - *Explore data*: What relationships are present in the data? How do we visualize them?
2. Data Preparation
    - *Select / exclude data*: What features and records will we keep or remove? Why?
    - *Clean data*: What values do we need to add, remove, or alter from when we verified data quality?
    - *Format data*: Are the datatypes accurate? Can they be changed? Which are the most useful to our model?
    - *Feature engineering*: Can we introduce better features, or consolidate the features we have to reduce dimensionality?
3. Modeling
    - *Select modeling techniques*: What models will we try?
    - *Determine test design*: How will we test our model?
    - *Build model*: How do we construct our model?
    - *Tune model*: How do we change our model to be more accurate?
    - *Assess model*: What are the results of our tests?
4. Evaluation
    - *Evaluate model result*: Does our model meet the stakeholder's requirements?
    - *Review process*: Did we miss anything? What else should be added?
    - *Determine next steps*: Should we deploy the model, or iterate further?
5. Deployment
    - We expect the stakeholder will want to make this model available for use beyond our project. We will need to create a way for users to provide data to our model, and retrieve its predictions.

The plan is not intended to be strictly sequential - rather, it is intended to be flexible to allow us to return to previous sections to re-evaluate our approach when new information arises or project circumstances change.

## Data Understanding

Before we can manipulate our data or build a model from it, we must understand it.

Let's load our data from the CSV files.

In [2]:
# load data into dataframe
df = pd.merge(
    pd.read_csv('train_labels.csv').set_index('id'),
    pd.read_csv('train_features.csv').set_index('id'),
    left_index=True,
    right_index=True
)

### Split Data

The Kaggle competition originally created for this project had its own private dataset. Since we do not have access to this dataset, we will need to create a testing set from the available data. We will need to know how many observations we have before splitting, so both our training and testing sets have a healthy amount of data.

In [4]:
print(f"Our dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

Our dataset has 47520 rows and 40 columns.


We should also see what percentage of our dataset each class composes. If the classes are uneven, we will need to perform a stratified split of our data to ensure that our testing dataset has a congruent proportion of each class.

In [28]:
print("The proportion of values in each class is: ")
print(df.status_group.value_counts(normalize=True).to_string())

The proportion of values in each class is: 
functional                 0.542971
non functional             0.384091
functional needs repair    0.072938


Our classes are imbalanced, especially for 'functional needs repair.' Given that this class comprises only 7% of the data, we will need to be careful about how to split our data. We want as many records of this class in our training set as possible so we can create an accurate model, but we want enough records of this class in our testing set to provide meaningful feedback.

Given our near 50,000 records, we should aim to have at least 500 records for the 'functional needs repair' class in our training set. With 500 records, each record in this class will account for 0.2% of the class' prediction accuracy.

To find our optimal split percentage:

$$ 47520 * split_{pct} * 0.0729 = 500 $$
$$ split_{pct} = \frac{500}{47520 * 0.0729} $$
$$ split_{pct} = 0.144 $$

Let's make our testing set an even 15% of our total data.

In [29]:
# split training and testing sets
testing_set = df.groupby('status_group', group_keys=False).apply(lambda x: x.sample(frac=0.15))
training_set = df.drop(testing_set.index, axis=0)

# validate split
print(f"Our training set has {training_set.shape[0]} rows and our testing set has {testing_set.shape[0]} rows.")
print()
print("The proportion of classes in our training set is: ")
print(training_set.status_group.value_counts(normalize=True).to_string())
print()
print("The proportion of classes in our testing set is: ")
print(testing_set.status_group.value_counts(normalize=True).to_string())
print()
print("The number of 'functional needs repair' records in our testing set is: ")
print(testing_set.status_group.value_counts(ascending=True)[0])


Our training set has 40392 rows and our testing set has 7128 rows.

The proportion of classes in our training set is: 
functional                 0.542979
non functional             0.384086
functional needs repair    0.072935

The proportion of classes in our testing set is: 
functional                 0.542929
non functional             0.384119
functional needs repair    0.072952

The number of 'functional needs repair' records in our testing set is: 
520


We have separated a testing set to test the accuracy of our model. Now let's start looking at the data in our training set.

### Data Source

We've recieved our data in a CSV format from Kaggle, but we should still ask the question, *where does our data come from?*

From the Kaggle competition description:
```
The data comes from the Taarifa waterpoints dashboard, which aggregates data from the Tanzania Ministry of Water. In their own words:

Taarifa is an open-source platform for the crowd-sourced reporting and triaging of infrastructure-related issues. Think of it as a bug tracker for the real world which helps to engage citizens with their local government.
```

So the data is crowd-sourced, perhaps by the citizens using the pumps, or by volunteers. Either way, there are some questions which remain unanswered by the description:
- *Who inputs the data into the dashboard?* Is it the people who use the pumps, volunteers, etc?
- *Are datatypes altered between entry and dashboarding?* Has the data changed in the pipeline between user entries and the CSV we received? Is there a potential for information to be lost in this process?
- *Who verifies the accuracy of the data?* Does the Ministry of Water verify, or does it rely on crowd-sourcing to correct errors?
- *What data is updated, and what data is static?* What data is inputted by dashboard users, and what data remains unaltered since the pump's creation?

In leiu of answers to these questions, we will need to make our best educated guesses about the data we're working with.

### Identify Data Surface Qualities

We've already split our data into training and testing sets, and from that, we know that we have 40392 records to train our model, and we're working with 39 features (40, less the dependent variable 'status group') Let's take another look at the breakdown of classes in the records, and this time, we'll also include the record counts for each class.

In [30]:
print("The proportion of classes in our training set is: ")
print(training_set.status_group.value_counts(normalize=True).to_string())
print()
print("The record counts of classes in our training set is: ")
print(training_set.status_group.value_counts().to_string())

The proportion of classes in our training set is: 
functional                 0.542979
non functional             0.384086
functional needs repair    0.072935

The record counts of classes in our training set is: 
functional                 21932
non functional             15514
functional needs repair     2946


Let's also take a look at the datatype counts, and our datatypes for each feature.

In [37]:
print("The counts for each datatype are: ")
print(training_set.dtypes.value_counts().to_string())

The counts for each datatype are: 
object     31
int64       6
float64     3


In [39]:
print("The datatypes for each feature are: ")
print(training_set.dtypes.to_string())

The datatypes for each feature are: 
status_group              object
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
paymen

The 'object' datatype can take multiple forms, the most common being string data. However, just because the datatype is 'object' does not mean that all of the data in the column is string data. It is possible for an 'object' datatype to contain multiple datatypes. Our data is likely dirty and needs to be cleaned.

### Verify Data Quality

Given that we know our data might be dirty, this is a good time to see how many null (nonexistent) values there are in the data.

In [40]:
print(f"There is a total of {training_set.isnull().sum().sum()} null values in the dataset.")

There is a total of 31346 null values in the dataset.


In [41]:
print("The counts of null values by feature are: ")
print(training_set.isnull().sum().to_string())

The counts of null values by feature are: 
status_group                 0
amount_tsh                   0
date_recorded                0
funder                    2442
gps_height                   0
installer                 2457
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 257
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2273
recorded_by                  0
scheme_management         2650
scheme_name              19186
permit                    2081
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type               

We have plenty of null values in our data, the most egregious omission being in the 'scheme_name' feature. We will need to determine the importance of this column, as we will likely drop it in the cleaning process because there is so much data missing.

It is important to note that these are only the *explicit* null values, or values explicitly left blank. We will also need to be vigilant in finding *implicit* null values - values which appear as valid in our dataset, but are intended as nulls (such as an 'x' in a field where a string name should be). To identify these possible implicit nulls, we will need to better understand the features we're working with so we can determine which values are valid.

### Understand Features

We have 39 features in our dataset. Let's see what forms they take, and do the research necessary to understand them. This will enable us to achieve better results in the remaining analysis, cleaning, and modeling.